In [1]:
import sys
src_path = "/Users/kimathikaai/workspace/fond/"
if src_path not in sys.path:
    sys.path.insert(0, src_path)
print(sys.path)

['/Users/kimathikaai/workspace/fond/', '/Users/kimathikaai/workspace/fond/src/utils', '/opt/homebrew/Cellar/python@3.11/3.11.4_1/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/opt/homebrew/Cellar/python@3.11/3.11.4_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/opt/homebrew/Cellar/python@3.11/3.11.4_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/kimathikaai/workspace/envs/fond/lib/python3.11/site-packages']


In [14]:
import argparse

import wandb
import yaml
import copy
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.train.fit import fit
from src.utils.hparams import random_hparams, seed_hash
from src.utils.run_info import get_project_runs, find_best_steps

%matplotlib inline
%load_ext autoreload
%autoreload 2

ENTITY = 'critical-ml-dg'
PROJECT_NAME = 'classes'
SWEEP_IDS = ['518qlq2m', '0alcovf9', '2suqerrf', '5gjc78bl']
# FILTERING_CRITERIA = {'algorithm':'ERM'}
FILTERING_CRITERIA = {}
METRIC_NAME = 'val/nacc'
TEST_METRIC = 'test/nacc'
METRIC_GOAL = 'max'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
# setup api
api = wandb.Api(
    overrides={
        "entity": ENTITY,
    },
    timeout=29
)

# get runs
runs, unique_datasets = get_project_runs(
    api_conn=api,
    project_name=PROJECT_NAME,
    sweep_ids=SWEEP_IDS,
    filtering_criteria=FILTERING_CRITERIA,
)

[info] Extracting sweep information from the following sweeps: ['518qlq2m', '0alcovf9', '2suqerrf', '5gjc78bl']


100%|████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 84.08it/s]

[info] Runs to process: 240


In [16]:
# visualize a run
# type(runs[0])
# hist = list(runs[0].scan_history(keys=['val/nacc', 'step']))
runs[0]

<Run critical-ml-dg/classes/vkqhjnki (finished)>

In [17]:
# visualize a run metric
list(runs[0].scan_history(keys=['val/nacc', 'step']))[-1]

{'val/nacc': 0.28663251797358197, 'step': 5000}

In [18]:
best_step_info = find_best_steps(
    runs=runs, 
    val_metric=METRIC_NAME, 
    metric_goal=METRIC_GOAL, 
    test_metric=TEST_METRIC
)

100%|██████████████████████████████████████████████████████| 240/240 [13:21<00:00,  3.34s/it]


In [19]:
key = list(best_step_info.keys())[0]
key, best_step_info[key]

(<Run critical-ml-dg/classes/vkqhjnki (finished)>,
 {'val/nacc': 0.3015453616778056,
  'step_number': 2700,
  'test/nacc': 0.5724503993988037})

In [20]:
# Runs are organized based on dataset, overlap and test_id
# dataset -> num_domain_linked_classes -> num_classes -> test_domain_id -> [hparam_id]
# Note: Performing only one trial
INFO = {
    dataset:{
        num_domain_linked_classes:{
            num_classes:{
                test_domain_id: {
                    'run_id': None, 
                    'step': None,
                    METRIC_NAME: float("inf") if METRIC_GOAL == "min" else float("-inf"),
                    TEST_METRIC: None
                } for test_domain_id in range(4)
            } for num_classes in range(5,70, 5)
        } for num_domain_linked_classes in range(5,70,5)
    } 
    for dataset in ['PACS', 'VLCS', 'OfficeHome']
}
algorithm_info = {}
for run, info in tqdm(best_step_info.items()):
    # Get run infomation
    run_id = run.id
    run_dataset = run.config["dataset"]
    run_test_domain_id = run.config["test_domain_id"]
    run_num_domain_linked_classes = run.config['num_domain_linked_classes']
    run_num_classes = run.config['num_classes']
    run_algorithm = run.config['algorithm']
    # get historical data
    run_val = info[METRIC_NAME]
    run_step = info['step_number']
    run_test = info[TEST_METRIC]

    # create entry 
    if run_algorithm not in algorithm_info:
        algorithm_info[run_algorithm] = copy.deepcopy(INFO)
        algorithm_info[
            run_algorithm][
            run_dataset][
            run_num_domain_linked_classes][
            run_num_classes][
            run_test_domain_id] = {
                "run_id": run_id, 
                METRIC_NAME: run_val, 
                'step': run_step,
                TEST_METRIC: run_test
            }
        # print(algorithm_info)
        continue

    # get previous validation value
    previous_val = algorithm_info[
            run_algorithm][
            run_dataset][
            run_num_domain_linked_classes][
            run_num_classes][
            run_test_domain_id][METRIC_NAME]

    # compare based on goal
    if (METRIC_GOAL=="min") and (previous_val>run_val):
        algorithm_info[
            run_algorithm][
            run_dataset][
            run_num_domain_linked_classes][
            run_num_classes][
            run_test_domain_id] = {
                "run_id": run_id, 
                METRIC_NAME: run_val, 
                'step': run_step,
                TEST_METRIC: run_test
            }
    elif (METRIC_GOAL=="max") and (previous_val<run_val):
        algorithm_info[
            run_algorithm][
            run_dataset][
            run_num_domain_linked_classes][
            run_num_classes][
            run_test_domain_id] = {
                "run_id": run_id, 
                METRIC_NAME: run_val, 
                'step': run_step,
                TEST_METRIC: run_test
            }

algorithm_info    

100%|███████████████████████████████████████████████████| 240/240 [00:00<00:00, 16196.57it/s]


{'ERM': {'PACS': {5: {5: {0: {'run_id': None,
      'step': None,
      'val/nacc': -inf,
      'test/nacc': None},
     1: {'run_id': None, 'step': None, 'val/nacc': -inf, 'test/nacc': None},
     2: {'run_id': None, 'step': None, 'val/nacc': -inf, 'test/nacc': None},
     3: {'run_id': None, 'step': None, 'val/nacc': -inf, 'test/nacc': None}},
    10: {0: {'run_id': None,
      'step': None,
      'val/nacc': -inf,
      'test/nacc': None},
     1: {'run_id': None, 'step': None, 'val/nacc': -inf, 'test/nacc': None},
     2: {'run_id': None, 'step': None, 'val/nacc': -inf, 'test/nacc': None},
     3: {'run_id': None, 'step': None, 'val/nacc': -inf, 'test/nacc': None}},
    15: {0: {'run_id': None,
      'step': None,
      'val/nacc': -inf,
      'test/nacc': None},
     1: {'run_id': None, 'step': None, 'val/nacc': -inf, 'test/nacc': None},
     2: {'run_id': None, 'step': None, 'val/nacc': -inf, 'test/nacc': None},
     3: {'run_id': None, 'step': None, 'val/nacc': -inf, 'test/nacc'

In [21]:
algorithm_info['ERM']['OfficeHome'][5][5]

{0: {'run_id': 'accmhclg',
  'val/nacc': 1,
  'step': 2700,
  'test/nacc': 0.842220664024353},
 1: {'run_id': 'roueijnj',
  'val/nacc': 1,
  'step': 900,
  'test/nacc': 0.7793834209442139},
 2: {'run_id': 'mo30hwzc',
  'val/nacc': 0.9818181991577148,
  'step': 900,
  'test/nacc': 0.7590621709823608},
 3: {'run_id': '42f4n3bs',
  'val/nacc': 0.9777777791023254,
  'step': 900,
  'test/nacc': 0.9203030467033386}}

In [22]:
# CREATE TABLE
# For a given algorithm and dataset create a table
# Create a table -> algorithm, dataset, num_domain_linked_classes, num_classes, test_domain_id, test/nacc
# Note we will need to average
table = []
for algo in algorithm_info:
    for dataset in algorithm_info[algo]:
        for num_domain_linked_classes in algorithm_info[algo][dataset]:
            for num_classes in algorithm_info[algo][dataset][num_domain_linked_classes]:
                for test_domain_id in algorithm_info[algo][dataset][num_domain_linked_classes][num_classes]:
                    row = {
                        'algorithm': algo,
                        'dataset': dataset,
                        'num_domain_linked_classes': num_domain_linked_classes,
                        'num_domain_shared_classes': num_classes - num_domain_linked_classes,
                        'num_classes': num_classes,
                        'test_domain_id': test_domain_id,
                        TEST_METRIC: algorithm_info[algo][dataset][num_domain_linked_classes][num_classes][test_domain_id][TEST_METRIC]
                    }
                    table.append(row)
df = pd.DataFrame(table)
df

,algorithm,dataset,num_domain_linked_classes,num_domain_shared_classes,num_classes,test_domain_id,test/nacc
0,ERM,PACS,5,0,5,0,NaN
1,ERM,PACS,5,0,5,1,NaN
2,ERM,PACS,5,0,5,2,NaN
3,ERM,PACS,5,0,5,3,NaN
4,ERM,PACS,5,5,10,0,NaN
...,...,...,...,...,...,...,...
4051,FOND,OfficeHome,65,-5,60,3,NaN
4052,FOND,OfficeHome,65,0,65,0,NaN
4053,FOND,OfficeHome,65,0,65,1,NaN
4054,FOND,OfficeHome,65,0,65,2,NaN


In [25]:
df.to_csv('2024-02-26.csv', index=False)

In [27]:
df = pd.read_csv('2024-02-26.csv')
_df = df.groupby(['test_domain_id','algorithm', 'dataset', 'num_domain_linked_classes', 'num_domain_shared_classes', 'num_classes']).mean(numeric_only=True)
_df.loc[_df[TEST_METRIC].notnull()]

test/nacc
test_domain_id algorithm dataset    num_domain_linked_classes num_domain_shared_classes num_classes           
0              ERM       OfficeHome 5                         0                         5             0.842221
                                                              10                        15            0.774446
                                                              20                        25            0.751285
                                    15                        0                         15            0.724528
                                                              20                        35            0.659103
                                                              50                        65            0.660338
                                    40                        0                         40            0.534827
                                                              15                        55            0.525557
                                                              25                        65            0.527655
               FOND      OfficeHome 5                         0                         5             0.874595
                                                              10                        15            0.803340
                                                              20                        25            0.835015
1              ERM       OfficeHome 5                         0                         5             0.779383
                                                              10                        15            0.412515
                                                              20                        25            0.360656
                                    15                        0                         15            0.605100
                                                              20                        35            0.379844
                                                              50                        65            0.331912
                                    40                        0                         40            0.435314
                                                              15                        55            0.337887
                                                              25                        65            0.265291
               FOND      OfficeHome 5                         0                         5             0.694704
                                                              10                        15            0.406012
                                                              20                        25            0.415219
2              ERM       OfficeHome 5                         0                         5             0.759062
                                                              10                        15            0.410948
                                                              20                        25            0.346115
                                    15                        0                         15            0.676290
                                                              20                        35            0.363983
                                                              50                        65            0.250736
                                    40                        0                         40            0.564751
                                                              15                        55            0.351599
                                                              25                        65            0.287668
               FOND      OfficeHome 5                         0                         5             0.808523
                                                              10                        15            0.4